# Generate SMA using daily data service - F#

### Overview
Moving averages smooth the price movements by filtering out the "noise" caused by short-term price fluctuations.They also form the building blocks for a wide range of technical indicators.

Computing moving averages over a defined number of time periods is useful to:
* identify the trend direction
* determine support and resistance levels

This sample demonstrates how to request and plot **on-demand** *simple moving average - SMA* from a daily data service.

### Inputs/outputs
Simple moving averages' requests require instrument's identifier, date time intervals and number of periods as per inputs.

Basically, SMA is formed by computing the average price of a security over a specific number of points. As its name implies, simple moving average is an average that moves. Old data is dropped and new data becomes available, causing the average to move along the time.

N-points SMA data point is computed by adding up the last N period's prices and then dividing by N as follows:

$$ SMA = \frac{\Sigma Close_n}{N} $$
Where:
* *$Close_n$* is the close price of the *period n*
* *N* is the count of time periods

This sample shows how to plot a simple graph for basis technical analysis using an open source library. 

### Services used
This sample uses *gRPC requests* in order to retrieve daily data from the hosted service. The queried endpoint in this script are:
* *DailySmaService*: to directly retrieve daily SMA data from the server

### Packages required
1. Systemathics packages:
    * *Systemathics.Apis.Type.Shared.V1*
    * *Systemathics.Apis.Services.DailyAnalytics.V1*
2. Open source packages
    * *Google.Protobuf.WellKnownTypes*
    * *Google.Type*
    * *Grpc.Net.Client*
    * *Grpc.Core*
    * *XPlot.Plotly* as per display package

***

# Run daily SMA sample

### Step 1: Install packages

In [ ]:
#i "nuget: file:///home/jovyan/.nuget/packages/"
#i "nuget: https://api.nuget.org/v3/index.json"
#r "nuget: Systemathics.Apis"
#r "nuget: XPlot.Plotly.Interactive"

In [ ]:
open Systemathics.Apis.Helpers
open Systemathics.Apis.Type.Shared.V1
open Systemathics.Apis.Services.DailyAnalytics.V1
open Google.Protobuf.WellKnownTypes
open Google.Type
open Grpc.Net.Client
open Grpc.Core
open XPlot.Plotly
open XPlot.Plotly.Interactive

### Step 2: Prepare API requests
The following code snippets retrieve authentication token and prepare the API request by: opening the *channel* and adding the *token* to the request header:

In [ ]:
// Get token as metadata
let headers = TokenHelpers.GetTokenAsMetaData();

// Create communication channel
let channel = ChannelHelpers.GetChannel();

### Step 3: Retrieve data

To request *daily sma* service, we need to specify:
* Instrument identifier
* Time period selection: select start and end dates
* The SMA parameters

#### 3.1 Instrument selection

In [ ]:
// Set instrument
let ticker = "AAPL"
let exchange = "XNGS"

#### 3.2 SMA parameters

In [ ]:
// Set SMAs lengths
let longSma = 200
let shortSma = 50

#### 3.3 Time period selection

In [ ]:
// Set data interval (we are using Google date time format)
let dateIntervals = 
  new DateInterval(
    StartDate = new Date ( Year = 2019, Month = 01, Day = 01 ),
    EndDate = new Date ( Year = 2020, Month = 12, Day = 31 )
  )

// Generate constraints based on the previous date selection
// Time constraints are not neeeded for the splits
let constraints = new Constraints() 
constraints.DateIntervals.Add(dateIntervals)

#### 3.4 Create and process request
The following code snippet creates *gRPC client*, process daily SMA requests and returns the reply.

In [ ]:
// Instantiate the daily SMA service
let service = new DailySmaService.DailySmaServiceClient(channel)

In [ ]:
// Create and process the daily long SMA request
let dailyLongSmaRequest = 
    new DailySmaRequest( 
        Identifier = new Identifier ( Exchange = exchange, Ticker = ticker ), 
        Length = longSma,
        Constraints = constraints
    )
let dailyLongSmaReply = service.DailySma(dailyLongSmaRequest, headers)

// Create and process the daily short SMA request
let dailyShortSmaRequest = 
    new DailySmaRequest ( 
        Identifier = new Identifier ( Exchange = exchange, Ticker = ticker ), 
        Length = shortSma,
        Constraints = constraints
    )
let dailyShortSmaReply = service.DailySma(dailyShortSmaRequest, headers)

// Visualize requests' results
//dailyLongSmaReply.Data.Count |> display
//dailyShortSmaReply.Data.Count |> display

### Step 4: Visualize data

#### 4.1 Select data

In [ ]:
// Extract the price and the SMAs
let prices = dailyLongSmaReply.Data |> Seq.map (fun t -> (new DateTime(t.Date.Year, t.Date.Month, t.Date.Day), t.Value))
let longSmaData = dailyLongSmaReply.Data |> Seq.map (fun t -> (new DateTime(t.Date.Year, t.Date.Month, t.Date.Day), t.Average))
let shortSmaData = dailyShortSmaReply.Data |> Seq.map (fun t -> (new DateTime(t.Date.Year, t.Date.Month, t.Date.Day), t.Average))

#### 4.2 Plot SMAs and daily prices
Plot SMA request results with the package of your choice.<br>*`XPlot.Plotly`* is used as per open source display package. 

In [ ]:
[
    new Scattergl(
        name = $"{longSma}-days SMA",
        mode = "lines",
        x = (longSmaData |> Seq.map fst),
        y = (longSmaData |> Seq.map snd)
    )
    new Scattergl(
        name = $"{shortSma}-days SMA",
        mode = "lines",
        x = (shortSmaData |> Seq.map fst),
        y = (shortSmaData |> Seq.map snd)
    )
    new Scattergl(
        name = "Price",
        mode = "lines",
        x = (prices |> Seq.map fst),
        y = (prices |> Seq.map snd)
    )
]
|> Chart.Plot
|> Chart.WithTitle $"Long & short SMAs and daily prices for {ticker}"
|> Chart.WithXTitle "Date"
|> Chart.WithWidth 1500
|> Chart.WithHeight 800
|> display